# RAG

ML環境構築

1. Amazon BedrockとのMLコネクターを作成
1. モデルグループを作成
1. モデルを作成
1. モデルをデプロイ
1. Searchパイプラインを作成

インデックス作成

1. インデックスを作成
1. データを登録

検索

1. RAG検索


---
## OpenSearch 起動・停止

* 起動

In [2]:
%%bash
docker compose -f "../0_opensearch-docker/docker-compose.yml" up -d


 Network 0_opensearch-docker_opensearch-net  Creating
 Network 0_opensearch-docker_opensearch-net  Created
 Container opensearch-dashboards  Creating
 Container opensearch-node1  Creating
 Container opensearch-dashboards  Created
 Container opensearch-node1  Created
 Container opensearch-node1  Starting
 Container opensearch-dashboards  Starting
 Container opensearch-node1  Started
 Container opensearch-dashboards  Started


* 停止

In [ ]:
%%bash
docker compose -f "../0_opensearch-docker/docker-compose.yml" down
# docker compose -f "../0_opensearch-docker/docker-compose.yml" down -v


---
## 事前準備


In [3]:
%pip install -Uq requests python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import requests


In [5]:
from dotenv import load_dotenv

load_dotenv(override=True)


True

In [6]:
host = 'localhost'
port = 9200


---
## 1. Amazon BedrockとのMLコネクターを作成


In [7]:
body = {
    "name": "Amazon Bedrock(anthropic.claude-instant-v1)",
    "description": "Test connector for Amazon Bedrock",
    "version": 1,
    "protocol": "aws_sigv4",
    "credential": {
      "access_key": os.getenv('AWS_ACCESS_KEY_ID'),
      "secret_key": os.getenv('AWS_SECRET_ACCESS_KEY')
    },
    "parameters": {
      "region": os.getenv('AWS_DEFAULT_REGION'),
      "service_name": "bedrock"
    },
    "actions": [
      {
        "action_type": "predict",
        "method": "POST",
        "headers": {
          "content-type": "application/json"
        },
            "url": "https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-instant-v1/invoke",
            "request_body": "{\"prompt\":\"\\n\\nHuman: ${parameters.inputs}\\n\\nAssistant:\",\"max_tokens_to_sample\":2000,\"temperature\":0.5,\"top_k\":250,\"top_p\":1,\"stop_sequences\":[\"\\\\n\\\\nHuman:\"]}"
      }
    ]
}

response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/connectors/_create', 
  json=body
  )

response.json()


{'connector_id': '-TWfw4sBcYKsbL--Z6qv'}

In [8]:
connector_id = response.json()["connector_id"]
connector_id


'-TWfw4sBcYKsbL--Z6qv'

---
## 2. モデルグループを作成


In [9]:
model_group_name = "bedrock-claude-instant-model-group"

body = {
    "name": model_group_name
}

response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/model_groups/_register', 
  json=body
  )

response.json()


{'model_group_id': '-jWgw4sBcYKsbL--AKrR', 'status': 'CREATED'}

In [10]:
model_group_id = response.json()["model_group_id"]
model_group_id


'-jWgw4sBcYKsbL--AKrR'

---
## 3. モデルを作成


In [11]:
model_name = "bedrock-claude-instant-model"

body = {
    "name": model_name,
    "function_name": "remote",
    "model_group_id": model_group_id,
    "connector_id": connector_id
}

response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/models/_register', 
  json=body
  )

response.json()


{'task_id': '-zWgw4sBcYKsbL--jqpy',
 'status': 'CREATED',
 'model_id': '_DWgw4sBcYKsbL--j6qL'}

In [12]:
model_id = response.json()["model_id"]
model_id


'_DWgw4sBcYKsbL--j6qL'

---
## 4. モデルをデプロイ


In [13]:
response = requests.post(
  f'http://{host}:{port}/_plugins/_ml/models/{model_id}/_deploy', 
  )

response.json()


{'task_id': '_TWgw4sBcYKsbL--6Kpo',
 'task_type': 'DEPLOY_MODEL',
 'status': 'COMPLETED'}

---
## 5. RAGフィーチャーフラグを有効化
 

In [14]:
body = {
  "persistent": {
    "plugins.ml_commons.rag_pipeline_feature_enabled": "true"
    }
}

response = requests.put(
  f'http://{host}:{port}/_cluster/settings', 
  json=body
  )

response.json()


{'acknowledged': True,
 'persistent': {'plugins': {'ml_commons': {'rag_pipeline_feature_enabled': 'true'}}},
 'transient': {}}

---
## 6. Searchパイプラインを作成


In [15]:
search_pipeline_name = 'rag-search-pipeline'

body = {
  "response_processors": [
    {
      "retrieval_augmented_generation": {
        "model_id": model_id,
        "context_field_list": ["answer"],
        "system_prompt": "You are a helpful assistant",
        "user_instructions": "Generate a concise and informative answer in less than 100 words for the given question"
      }
    }
  ]
}

response = requests.put(
  f'http://{host}:{port}/_search/pipeline/{search_pipeline_name}', 
  json=body
  )

response.json()


{'acknowledged': True}

---
## 1. インデックスを作成


In [17]:
index_name = 'rag-index'


In [18]:
body = {
  "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "custom_kuromoji_analyzer": {
            "tokenizer": "kuromoji_tokenizer",
            "filter": ["kuromoji_baseform", "ja_stop"],
            "char_filter": ["icu_normalizer"]
          }
        }
      }
    }
  }, 
  "mappings": {
    "properties": {
      "question": {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
      "answer": {"type": "text", "analyzer": "custom_kuromoji_analyzer"}
    }
  }
}

response = requests.put(
  f"http://{host}:{port}/{index_name}", 
  json=body
  )

response.json()


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'rag-index'}

---
## 2. データを登録

In [19]:
%pip install -Uq langchain beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter

loader = WebBaseLoader("https://aws.amazon.com/jp/ec2/faqs/")
data = loader.load()

text_splitter = CharacterTextSplitter(
    separator = "Q:",
    keep_separator=True,
    chunk_size = 10,
    chunk_overlap  = 0,
)

# 先頭のいらないものを消す
texts = text_splitter.split_documents(data)
texts = texts[1:]
# 末尾のいらないものを消す
texts[-1].page_content = texts[-1].page_content.split('\xa0')[0]


Created a chunk of size 1283, which is longer than the specified 10
Created a chunk of size 154, which is longer than the specified 10
Created a chunk of size 425, which is longer than the specified 10
Created a chunk of size 275, which is longer than the specified 10
Created a chunk of size 190, which is longer than the specified 10
Created a chunk of size 582, which is longer than the specified 10
Created a chunk of size 832, which is longer than the specified 10
Created a chunk of size 483, which is longer than the specified 10
Created a chunk of size 203, which is longer than the specified 10
Created a chunk of size 826, which is longer than the specified 10
Created a chunk of size 211, which is longer than the specified 10
Created a chunk of size 574, which is longer than the specified 10
Created a chunk of size 322, which is longer than the specified 10
Created a chunk of size 199, which is longer than the specified 10
Created a chunk of size 365, which is longer than the specifi

In [21]:
for text in texts:
  try:
    lines = text.page_content.splitlines()
    q = lines[0]
    a = text.page_content

    body = {
      "question": q,
      "answer": a
    }

    response = requests.post(
      f'http://{host}:{port}/{index_name}/_doc', 
      json=body
    )
  except Exception as e:
    print(e)


---
## 1. RAG検索


In [23]:
question = "EC2で起動できるOSは何ですか？"


In [26]:
body = {
  "query": {
    "match": {
      "question": question
    }
  },
  "ext": {
		"generative_qa_parameters": {
      "llm_model": "bedrock/anthropic-instant", # 必ずbedrock/で始めること
			"llm_question": question,
      "context_size": 10
		}
	}
}

params = {
  "search_pipeline": search_pipeline_name,
  "timeout": "30s"
}

response = requests.get(
  f"http://{host}:{port}/{index_name}/_search", 
  json=body,
  params=params
  )

response.json()


{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 205, 'relation': 'eq'},
  'max_score': 6.3473625,
  'hits': [{'_index': 'rag-index',
    '_id': '_zWlw4sBcYKsbL--rqoS',
    '_score': 6.3473625,
    '_source': {'question': 'Q: Amazon EC2 で何ができますか?',
     'answer': 'Q: Amazon EC2 で何ができますか?\nAmazon Simple Storage Service (Amazon S3) がクラウド内のストレージを可能とするのとまったく同様に、Amazon EC2 は、クラウド内での「コンピューティング」を可能にします。\xa0 Amazon EC2 のシンプルなウェブサービスインターフェイスによって、手間をかけず、必要な機能を取得および設定できます。お客様のコンピューティングリソースに対して、高機能なコントロールが提供され、Amazon の実績あるインフラストラクチャ上で実行できます。Amazon EC2 では、わずか数分間で新規サーバーインスタンスを取得して起動できるようになります。これにより、コンピューティング要件の変化に合わせて、すばやく容量をスケールアップおよびスケールダウンできます。実際に使用した分のみ料金が発生するため、Amazon EC2 はコンピューティングの経済性も変革します。'}},
   {'_index': 'rag-index',
    '_id': '9zWmw4sBcYKsbL--MKzk',
    '_score': 6.303764,
    '_source': {'question': 'Q:Nitro Hypervisor では、インスタンスの再起動と終了の EC2 API リクエストがどのように実装されますか?',
     'answer': 'Q:Nitro Hypervisor では

In [30]:
print(
  response.json()["ext"]["retrieval_augmented_generation"]["answer"]
)



 Amazon EC2では、以下の主なOSが起動できます。

- Linuxディストリビューション(Amazon Linux、Ubuntu、CentOSなど)
- Windowsサーバー
- macOS
- Oracle Linux
- SUSE Linux
- FreeBSD
- IBM AIX
- Oracle Solaris

EC2では、様々なリリースされたLinuxディストリビューションやWindowsサーバーをサポートしています。また、特定の用途向けにOracle Linux、FreeBSD、AIX、Solarisイメージも提供しています。最近では、macOSもサポートOSの一つとなっています。


---
## クリーンアップ

* インデックス削除


In [ ]:
response = requests.delete(
  f"http://{host}:{port}/{index_name}", 
  )

response.json()
